In [1]:
1+1

2

In [2]:
import mozo2 

import importlib
importlib.reload(mozo2)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from mozo import compare_and_combine_2, Diff, merge_with_features, num_to_range_categories
import pandas as pd
import numpy as np
np.random.seed(0)

import xgboost as xgb

from scipy import stats
import math 
import datetime 

from sklearn.metrics import r2_score, mean_absolute_error
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from functools import partial
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from sklearn.svm import SVC

from sklearn.metrics import r2_score, f1_score, mean_absolute_error, mean_squared_error

from scikitplot.estimators import plot_learning_curve

import time
from datetime import datetime


In [3]:
## modelowanie

In [4]:
# import warnings
# warnings.filterwarnings("ignore")

total = pd.read_csv('~/pml7/konkurs/output/newest_total_20201101_07_20.h5.csv')
total.info(verbose=True)

train_30 = pd.read_hdf('output/train_30.h5', 'data') 
test_car_ids = train_30['car_id'].values

train_70 = total[~total['car_id'].isin(test_car_ids) & ~total['price_value'].isna()]
train_30 = total[total['car_id'].isin(test_car_ids)]

print(len(total.index))
print(len(total[~total['price_value'].isna()].index))

print(train_70['price_value'].isna().sum())
print(train_30['price_value'].isna().sum())

train_100 = total[~total['price_value'].isna()]
print(train_100['price_value'].isna().sum())

global_min = total[~total['price_value'].isna()].price_value.min()

test = total[total['price_value'].isna()]
train = total[~total['price_value'].isna()]


/home/conda/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,86,161) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 218 columns):
 #   Column                                                  Dtype  
---  ------                                                  -----  
 0   breadcrumb                                              object 
 1   created_at                                              object 
 2   price_currency                                          object 
 3   price_details                                           object 
 4   price_value                                             float64
 5   seller_address                                          object 
 6   seller_name                                             object 
 7   seller_type                                             object 
 8   feature_czujniki_parkowania_przednie                    bool   
 9   feature_poduszka_powietrzna_chroniąca_kolana            bool   
 10  feature_kurtyny_powietrzne                             

160616
106494
0
0
0


In [5]:
start_vars = np.sort([
# 'created_at_unix', 'encoded_param_napęd_2', 'encoded_param_rodzaj_paliwa_2', 'encoded_param_skrzynia_biegów_2', 'feature_abs', 'feature_alufelgi', 'feature_asr__kontrola_trakcji_', 'feature_asystent_parkowania', 'feature_bluetooth', 'feature_cd', 'feature_czujniki_parkowania_przednie', 'feature_elektryczne_szyby_tylne', 'feature_elektrycznie_ustawiane_lusterka', 'feature_esp__stabilizacja_toru_jazdy_', 'feature_gniazdo_sd', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_dwustrefowa', 'feature_klimatyzacja_manualna', 'feature_radio_fabryczne', 'feature_tempomat', 'feature_światła_do_jazdy_dziennej', 'feature_światła_led', 'feature_światła_xenonowe', 'latitude', 'longitude', 'model_XGBRegressor_5856_36229_', 'new_param_kod_silnika_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_model_pojazdu_cat', 'new_param_napęd_cat', 'new_param_oferta_od_cat', 'new_param_vin_cat', 'new_param_wersja_cat', 'new_seller_name_cat', 'param_bezwypadkowy', 'param_ile_lat_temu_produkcja', 'param_kierownica_po_prawej__anglik_', 'param_leasing', 'param_liczba_miejsc_2', 'param_moc', 'param_pierwsza_rejestracja_date_na', 'param_pojemność_skokowa', 'param_przebieg', 'param_rok_produkcji', 'param_serwisowany_w_aso', 'param_tuning', 'param_uszkodzony', 'param_wartość_wykupu_2', 'param_zarejestrowany_w_polsce', 'price_currency_pln'
# 'created_at_unix', 'encoded_param_rodzaj_paliwa_2', 'encoded_param_typ_2', 'feature_abs', 'feature_asr__kontrola_trakcji_', 'feature_bluetooth', 'feature_cd', 'feature_czujniki_parkowania_przednie', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_gniazdo_sd', 'feature_hud__wyświetlacz_przezierny_', 'feature_isofix', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_manualna', 'feature_ogrzewanie_postojowe', 'feature_podgrzewane_tylne_siedzenia', 'feature_relingi_dachowe', 'feature_tapicerka_skórzana', 'feature_tapicerka_welurowa', 'feature_tempomat', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'feature_światła_xenonowe', 'latitude_na_mean', 'longitude', 'longitude_na_mean', 'model_8790', 'new_param_kolor_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_oferta_od_cat', 'new_param_opłata_początkowa_cat', 'new_param_rodzaj_paliwa_cat', 'new_param_wersja_cat', 'new_seller_name_cat', 'new_vin_flex_fuel_cat', 'new_vin_model_cat', 'new_vin_security_cat', 'param_bezwypadkowy', 'param_homologacja_ciężarowa', 'param_ile_lat_temu_produkcja', 'param_leasing', 'param_liczba_miejsc_2', 'param_liczba_pozostałych_rat_2', 'param_matowy', 'param_przebieg', 'param_rok_produkcji', 'param_tuning', 'param_uszkodzony', 'param_wartość_wykupu_2'
# 'model_8790', 'feature_bluetooth', 'created_at_unix', 'encoded_param_rodzaj_paliwa_2', 'encoded_param_typ_2', 'feature_asr__kontrola_trakcji_', 'feature_asystent_parkowania', 'feature_cd', 'feature_centralny_zamek', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_gniazdo_sd', 'feature_isofix', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_manualna', 'feature_ogrzewanie_postojowe', 'feature_podgrzewane_tylne_siedzenia', 'feature_relingi_dachowe', 'feature_tapicerka_skórzana', 'feature_tempomat', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'longitude', 'latitude', 'new_param_kolor_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_oferta_od_cat', 'new_param_opłata_początkowa_cat', 'new_param_wersja_cat', 'new_seller_name_cat', 'new_vin_flex_fuel_cat', 'new_vin_model_cat', 'new_vin_portrait_cat', 'new_vin_security_cat', 'ogloszenie_rok', 'param_bezwypadkowy', 'param_homologacja_ciężarowa', 'param_ile_lat_temu_produkcja', 'param_leasing', 'param_liczba_miejsc_2', 'param_liczba_pozostałych_rat_2', 'param_matowy', 'param_moc', 'param_perłowy', 'param_pierwsza_rejestracja_10_lat', 'param_przebieg', 'param_rok_produkcji', 'param_tuning', 'param_uszkodzony', 'param_vin_len'
'encoded_param_rodzaj_paliwa_2', 'feature_abs', 'feature_asr__kontrola_trakcji_', 'feature_bluetooth', 'feature_centralny_zamek', 'feature_elektrycznie_ustawiane_fotele', 'feature_elektrycznie_ustawiane_lusterka', 'feature_gniazdo_sd', 'feature_immobilizer', 'feature_isofix', 'feature_klimatyzacja_automatyczna', 'feature_klimatyzacja_manualna', 'feature_komputer_pokładowy', 'feature_ogrzewanie_postojowe', 'feature_podgrzewane_tylne_siedzenia', 'feature_poduszka_powietrzna_kierowcy', 'feature_radio_fabryczne', 'feature_szyberdach', 'feature_tapicerka_skórzana', 'feature_tempomat', 'feature_zmieniarka_cd', 'feature_światła_led', 'feature_światła_przeciwmgielne', 'model_8790', 'new_param_kolor_cat', 'new_param_liczba_pozostałych_rat_cat', 'new_param_marka_pojazdu_cat', 'new_param_model_pojazdu_cat', 'new_param_oferta_od_cat', 'new_param_opłata_początkowa_cat', 'new_param_wersja_cat', 'new_seller_name_cat', 'new_vin_flex_fuel_cat', 'new_vin_model_cat', 'new_vin_portrait_cat', 'new_vin_security_cat', 'ogloszenie_rok', 'param_bezwypadkowy', 'param_filtr_cząstek_stałych', 'param_homologacja_ciężarowa', 'param_ile_lat_temu_produkcja', 'param_liczba_pozostałych_rat_2', 'param_matowy', 'param_metalik', 'param_moc', 'param_perłowy', 'param_pierwsza_rejestracja_10_lat', 'param_pierwsza_rejestracja_ten_rok', 'param_przebieg', 'param_serwisowany_w_aso', 'param_tuning'
])

# 'model_8790' = car_price_log_Vladimir_categories_xgb.csv; 5637.56722; "opis": "Vladimir z kategorialnymi mae na train_30"

black_list = ['car_id', 'price_details', 'price_value', 'price_value_log', 'price_value_pln', 'price_value_pln_log_transl', 'new_cat_price_value',
#               *[x for x in total.columns if x.startswith('model_')],
              'model_XGBRegressor_2195_24982_po_hiperopcie_caly_train', #train_100, przeuczony
              'model_XGBRegressor_3293_22097_', #train_100
              'longitude_na_mean', 'latitude_na_mean', #gorsze w xbfir
              *total.columns[total.isnull().any()],
             ]

feats3 = total.select_dtypes(include=[np.number, np.bool]).columns
feats3 = [x for x in feats3 if x not in black_list]
feats3 = np.sort(feats3)

print(total[start_vars].info(verbose=True))

print(total[feats3].info(verbose=True))
print(total[feats3].columns[total[feats3].isnull().any()])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160616 entries, 0 to 160615
Data columns (total 51 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   encoded_param_rodzaj_paliwa_2            160616 non-null  int64  
 1   feature_abs                              160616 non-null  bool   
 2   feature_asr__kontrola_trakcji_           160616 non-null  bool   
 3   feature_bluetooth                        160616 non-null  bool   
 4   feature_centralny_zamek                  160616 non-null  bool   
 5   feature_elektrycznie_ustawiane_fotele    160616 non-null  bool   
 6   feature_elektrycznie_ustawiane_lusterka  160616 non-null  bool   
 7   feature_gniazdo_sd                       160616 non-null  bool   
 8   feature_immobilizer                      160616 non-null  bool   
 9   feature_isofix                           160616 non-null  bool   
 10  feature_klimatyzacja_automatyczn

In [ ]:
import contextlib
import warnings
warnings.filterwarnings("ignore")
import json

params = {    
    'n_jobs': 12, 

    'n_estimators': 90, 
    'learning_rate': 0.1, 
    'subsample': 0.5, 
    'colsample_bytree': 0.9,
    'max_depth': 10, 
}

mae_global_min = 2615.5414745879784
big_iter = 0

current_vars=start_vars
all_vars = feats3 #maksymalny set

while True:

    pairs_so_far = []

    last_added = None
    last_removed = None

    last_trial = False
    big_iter += 1
    print("START iter=" + str(big_iter) + ', mae_global_min=' + str(mae_global_min))
    
    while True:
        with contextlib.redirect_stdout(None):
            current_vars, last_added, last_removed, pairs_so_far, last_trial =  mozo2.mutate_rand_feature_pairs(current_vars, all_vars, pairs_so_far)
#         print('pairs_so_far: ' + json.dumps(pairs_so_far, ensure_ascii=False, indent=4))
        
        print('added: ' + last_added)
        print('removed: ' + last_removed)
        print('len: ' + str(len(current_vars)))
        
        mae, r2, model, model_cl, learning_curve_plot = mozo2.train_and_submit(train_70, train_30, total.loc[total['price_value'].isna(),], 
                     current_vars, global_min, 
                     xgb.XGBRegressor, params, digitize=None, subfolder='XGBRegressor_parami',
                     file_sufix = 'parami', opis='start=2615.5414745879784', 
                     learning_curve = False, 
                     add_model_column_min = -1, total = total, 
                     save_min=mae_global_min, kaggle_min=-1)

        if mae < mae_global_min:
            mae_global_min = mae
            print('Improvement: ' + last_removed + ' -> ' + last_added)
        else:
            current_vars = mozo2.replace_feature(current_vars, last_added, last_removed)
            print('Rollback: ' + last_added + ' -> ' + last_removed)
        print('=============================================' + str(mae_global_min)+ '=============================================')

        if last_trial == True: 
            print("END iter=" + str(big_iter) + ', mae_global_min=' + str(mae))
            break

START iter=1, mae_global_min=2615.5414745879784
added: feature_regulowane_zawieszenie
removed: param_pierwsza_rejestracja_ten_rok
len: 51
mea: 2736.2958549605396
r2: 0.9778230473086795
Rollback: feature_regulowane_zawieszenie -> param_pierwsza_rejestracja_ten_rok
=============================================2615.5414745879784=============================================
added: new_param_kod_silnika_cat
removed: feature_bluetooth
len: 51
mea: 2790.468993169075
r2: 0.9774035034321451
Rollback: new_param_kod_silnika_cat -> feature_bluetooth
=============================================2615.5414745879784=============================================
added: new_param_napęd_cat
removed: param_homologacja_ciężarowa
len: 51
mea: 2643.2360744804737
r2: 0.9794488149517424
Rollback: new_param_napęd_cat -> param_homologacja_ciężarowa
=============================================2615.5414745879784=============================================
added: feature_asystent_parkowania
removed: feature_tempom

mea: 2633.450583298941
r2: 0.9790786726994173
Rollback: feature_ogranicznik_prędkości -> encoded_param_rodzaj_paliwa_2
=============================================2615.5414745879784=============================================
added: created_at_unix
removed: ogloszenie_rok
len: 51
mea: 2733.8475702576943
r2: 0.9782001663735309
Rollback: created_at_unix -> ogloszenie_rok
=============================================2615.5414745879784=============================================
added: feature_światła_do_jazdy_dziennej
removed: feature_elektrycznie_ustawiane_fotele
len: 51
mea: 2636.8963332390467
r2: 0.9791776973462291
Rollback: feature_światła_do_jazdy_dziennej -> feature_elektrycznie_ustawiane_fotele
=============================================2615.5414745879784=============================================
added: feature_poduszka_powietrzna_chroniąca_kolana
removed: param_ile_lat_temu_produkcja
len: 51
mea: 2785.361747390311
r2: 0.9783032688667961
Rollback: feature_poduszka_powietrzn

mea: 2648.6941855362074
r2: 0.9786588648392457
Rollback: new_param_kod_silnika_cat -> new_seller_name_cat
=============================================2615.5414745879784=============================================
added: new_param_rodzaj_paliwa_cat
removed: new_param_wersja_cat
len: 51
mea: 2626.710090612554
r2: 0.9792054233299559
Rollback: new_param_rodzaj_paliwa_cat -> new_param_wersja_cat
=============================================2615.5414745879784=============================================
added: created_at_unix
removed: param_serwisowany_w_aso
len: 51
mea: 2732.897134107563
r2: 0.9774898037618474
Rollback: created_at_unix -> param_serwisowany_w_aso
=============================================2615.5414745879784=============================================
added: feature_wielofunkcyjna_kierownica
removed: ogloszenie_rok
len: 51
mea: 2728.24547087153
r2: 0.9784614522601682
Rollback: feature_wielofunkcyjna_kierownica -> ogloszenie_rok
===========================================

mea: 2635.445420504927
r2: 0.9793181601494528
Rollback: feature_gniazdo_usb -> feature_asr__kontrola_trakcji_
=============================================2615.5414745879784=============================================
added: encoded_param_napęd_2
removed: new_param_wersja_cat
len: 51
mea: 2734.7018377396885
r2: 0.978172595339235
Rollback: encoded_param_napęd_2 -> new_param_wersja_cat
=============================================2615.5414745879784=============================================
added: param_rok_produkcji
removed: param_metalik
len: 51
mea: 2648.3555097650747
r2: 0.9788062052942214
Rollback: param_rok_produkcji -> param_metalik
=============================================2615.5414745879784=============================================
added: feature_czujnik_martwego_pola
removed: param_przebieg
len: 51
mea: 2716.9832404598164
r2: 0.977469076213007
Rollback: feature_czujnik_martwego_pola -> param_przebieg
=============================================2615.5414745879784======

mea: 2752.1522265708295
r2: 0.9778852803606057
Rollback: feature_regulowane_zawieszenie -> new_param_kolor_cat
=============================================2615.5414745879784=============================================
added: feature_gniazdo_usb
removed: feature_bluetooth
len: 51
mea: 2635.627801680984
r2: 0.9793438710146611
Rollback: feature_gniazdo_usb -> feature_bluetooth
=============================================2615.5414745879784=============================================
added: ogloszenie_godzina
removed: param_pierwsza_rejestracja_ten_rok
len: 51
mea: 2643.410009778463
r2: 0.9787330091018329
Rollback: ogloszenie_godzina -> param_pierwsza_rejestracja_ten_rok
=============================================2615.5414745879784=============================================
added: encoded_param_napęd_2
removed: param_matowy
len: 51
mea: 2748.9532091817873
r2: 0.9779993869303318
Rollback: encoded_param_napęd_2 -> param_matowy
=============================================2615.54147458

...

mea: 2752.860541458653
r2: 0.9785724591879746
Rollback: new_param_rodzaj_paliwa_cat -> feature_elektrycznie_ustawiane_fotele
=============================================2604.46495618895=============================================
added: feature_tempomat_aktywny
removed: new_param_oferta_od_cat
len: 51
mea: 2712.6325293813143
r2: 0.9784726443466331
Rollback: feature_tempomat_aktywny -> new_param_oferta_od_cat
=============================================2604.46495618895=============================================
added: new_param_kod_silnika_cat
removed: new_param_marka_pojazdu_cat
len: 51
mea: 2620.2471448425217
r2: 0.9791856894244142
Rollback: new_param_kod_silnika_cat -> new_param_marka_pojazdu_cat
=============================================2604.46495618895=============================================
added: price_currency_pln
removed: feature_wielofunkcyjna_kierownica
len: 51
mea: 2803.979651496727
r2: 0.9780445784502815
Rollback: price_currency_pln -> feature_wielofunkcyjna_ki

mea: 2766.0946170286466
r2: 0.9778745490551026
Rollback: new_param_emisja_co2_cat -> feature_ogrzewanie_postojowe
=============================================2604.46495618895=============================================
added: feature_dach_panoramiczny
removed: encoded_param_rodzaj_paliwa_2
len: 51
mea: 2630.091805098785
r2: 0.9787818803900377
Rollback: feature_dach_panoramiczny -> encoded_param_rodzaj_paliwa_2
=============================================2604.46495618895=============================================
added: created_at_unix_na_mean
removed: feature_zmieniarka_cd
len: 51
mea: 2627.9343216897864
r2: 0.9788466264470561
Rollback: created_at_unix_na_mean -> feature_zmieniarka_cd
=============================================2604.46495618895=============================================
added: new_seller_wojewodzctwo_cat
removed: new_param_liczba_pozostałych_rat_cat
len: 51
mea: 2641.1706597228504
r2: 0.97869644601005
Rollback: new_seller_wojewodzctwo_cat -> new_param_liczba_po

mea: 2764.467961048746
r2: 0.977857603471819
Rollback: new_seller_address_cat -> feature_tapicerka_skórzana
=============================================2604.46495618895=============================================
added: feature_kurtyny_powietrzne
removed: new_seller_name_cat
len: 51
mea: 2708.81249982944
r2: 0.9784583388056036
Rollback: feature_kurtyny_powietrzne -> new_seller_name_cat
=============================================2604.46495618895=============================================
added: param_akryl__niemetalizowany_
removed: new_param_liczba_pozostałych_rat_cat
len: 51
mea: 2632.0079070941188
r2: 0.9789952840032363
Rollback: param_akryl__niemetalizowany_ -> new_param_liczba_pozostałych_rat_cat
=============================================2604.46495618895=============================================
added: feature_poduszka_powietrzna_chroniąca_kolana
removed: param_tuning
len: 51
mea: 2703.1393232621936
r2: 0.9783513041344813
Rollback: feature_poduszka_powietrzna_chroniąca_

mea: 2623.005293135336
r2: 0.9788884564268355
Rollback: feature_czujnik_deszczu -> feature_podgrzewane_tylne_siedzenia
=============================================2604.46495618895=============================================
added: feature_poduszki_boczne_przednie
removed: feature_isofix
len: 51
mea: 2633.4258634434505
r2: 0.9785517108445747
Rollback: feature_poduszki_boczne_przednie -> feature_isofix
=============================================2604.46495618895=============================================
added: encoded_seller_miasto_297
removed: new_param_opłata_początkowa_cat
len: 51
mea: 2724.8280906619025
r2: 0.9783885664373217
Rollback: encoded_seller_miasto_297 -> new_param_opłata_początkowa_cat
=============================================2604.46495618895=============================================
added: feature_cd
removed: new_vin_portrait_cat
len: 51
mea: 2729.656595210824
r2: 0.9785193446603113
Rollback: feature_cd -> new_vin_portrait_cat
=================================

mea: 2621.6725206852802
r2: 0.9790369694847435
Rollback: feature_czujniki_parkowania_przednie -> feature_klimatyzacja_manualna
=============================================2604.46495618895=============================================
added: encoded_param_typ_2
removed: model_8790
len: 51
mea: 8056.205465902138
r2: 0.9013585597633433
Rollback: encoded_param_typ_2 -> model_8790
=============================================2604.46495618895=============================================
added: feature_system_start_stop
removed: feature_centralny_zamek
len: 51
mea: 2629.684034475676
r2: 0.978915333302396
Rollback: feature_system_start_stop -> feature_centralny_zamek
=============================================2604.46495618895=============================================
added: feature_podgrzewane_przednie_siedzenia
removed: param_przebieg
len: 51
mea: 2713.5802603191346
r2: 0.9775102806645913
Rollback: feature_podgrzewane_przednie_siedzenia -> param_przebieg
=================================

In [70]:
from kaggle.api.kaggle_api_extended import KaggleApi
kaggle_api = KaggleApi()
kaggle_api.authenticate()

leaderboard = kaggle_api.competition_view_leaderboard('dw-car-price-prediction')
print(pd.DataFrame(leaderboard['submissions'])[['teamName', 'submissionDate', 'score']].head())

          teamName                submissionDate       score
0       adamskorek  2020-10-31T23:26:16.9833333Z  5156.58793
1           pelele  2020-10-31T19:46:31.2633333Z  5369.00636
2             mozo  2020-10-30T16:38:46.7333333Z  5466.72056
3  Adrian Mich***i       2020-10-31T12:30:08.39Z  5578.96222
4   Remek Półtorak  2020-10-31T19:09:20.1866667Z  5581.85197


In [38]:
# model = RandomForestRegressor(n_estimators=50, max_depth=7).fit(X_sub_train, y_sub_train) 
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.svm import SVC


# perm = PermutationImportance(model).fit(X_sub_test, y_sub_test) 
eli5.show_weights(perm, feature_names=random_feats, top=50) 


/home/conda/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/conda/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


NameError: name 'perm' is not defined

save_model(file_sufix='test', subfolder='', test = total.loc[total['price_value'].isna()],
               feats=current_vars, global_min=global_min, model=model, shift = 0, digitize=False, mae=mae, r2=r2,
               details='test zapisu', learning_curve_plot=None, hist_plt=None, kaggle=None)
